<a href="https://colab.research.google.com/github/Xornotor/Food101-CNN-Tensorflow/blob/main/Food101_CNN_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, SeparableConv2D, Conv2D
from tensorflow.keras.layers import Flatten, Dense, AvgPool2D, MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Normalization, Dropout
from tensorflow.keras.layers import Reshape, Resizing, Softmax
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import L2
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import tensorflow_datasets as tfds
from random import randint

from google.colab import drive
if(not os.path.exists('/content/drive')):
  drive.mount('/content/drive')

In [ ]:
train_ds, train_info = tfds.load('food101',
                            split='train',
                            with_info=True,
                            shuffle_files=True,
                            as_supervised=True,
                            batch_size=512
                          )

val_ds, val_info = tfds.load('food101',
                             split='validation',
                             with_info=True
                          )

In [ ]:
DROPOUT = 0.5
REGULARIZATION = 1e-4

def create_model():
  model = Sequential([
      Input((None, None, 3)),
      Resizing(256, 256, 'bilinear', True),
      Normalization(),
      SeparableConv2D(6, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(12, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      AvgPool2D((2, 2)),
      SeparableConv2D(24, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(32, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      AvgPool2D((2, 2)),
      SeparableConv2D(96, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(192, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      AvgPool2D((2, 2)),
      SeparableConv2D(384, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(512, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      AvgPool2D((2, 2)),
      Flatten(),
      Dense(256, activation='relu', kernel_regularizer=L2(REGULARIZATION)),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(192, activation='relu', kernel_regularizer=L2(REGULARIZATION)),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(128, activation='relu', kernel_regularizer=L2(REGULARIZATION)),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(100, activation='relu', kernel_regularizer=L2(REGULARIZATION)),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(100, activation='linear', kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      Softmax()
  ])

  model.compile(optimizer=Adam(learning_rate=3e-3),
                loss=SparseCategoricalCrossentropy(from_logits=False),
                metrics=[SparseCategoricalAccuracy(), F1Score()])
  
  return model

In [ ]:
FoodNet = create_model()
FoodNet.summary()

In [ ]:
#model_path = '/content/drive/MyDrive/Colab Notebooks/FoodNet/FoodNet.h5'
model_path = '/content/FoodNet.h5'

cp_callback = ModelCheckpoint(filepath=model_path,
                            save_weights_only=False,
                            verbose=1)

if(os.path.exists(model_path)):
    #FoodNet = load_model(model_path)
    pass

FoodNet.fit(train_ds,
            epochs=3,
            validation_data=val_ds,
            callbacks=[cp_callback])
